In [1]:

# !pip install torch scikit-learn --quiet

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score
import numpy as np

In [2]:

# Configuration
NUM_SAMPLES = 1000
SEQ_LEN = 10
NUM_DEPTS = 5
EMBED_DIM = 64
MASK_PROB = 0.15
NUM_SEASONS = 4

def generate_synthetic():
    dept_seq = torch.randint(0, NUM_DEPTS, (NUM_SAMPLES, SEQ_LEN))
    season_ids = torch.randint(0, NUM_SEASONS, (NUM_SAMPLES, SEQ_LEN))

    labels = torch.randint(0, 2, (NUM_SAMPLES, NUM_DEPTS)).float()

    customer_emb = torch.randn(NUM_SAMPLES, EMBED_DIM)
    product_emb = torch.randn(NUM_SAMPLES, SEQ_LEN, EMBED_DIM)
    season_emb = torch.randn(NUM_SAMPLES, EMBED_DIM)
    weather_emb = torch.randn(NUM_SAMPLES, EMBED_DIM)
    promo_emb = torch.randn(NUM_SAMPLES, EMBED_DIM)

    return {
        "sequences": dept_seq,
        "season_ids": season_ids,
        "labels": labels,
        "customer_emb": customer_emb,
        "product_emb": product_emb,
        "season_emb": season_emb,
        "weather_emb": weather_emb,
        "promo_emb": promo_emb
    }

data = generate_synthetic()


In [ ]:

def get_decay(seq_len, gamma=0.95):
    decay = torch.tensor([gamma**i for i in reversed(range(seq_len))], dtype=torch.float)
    decay = decay / decay.sum()
    return decay.unsqueeze(0)  # [1, T]


In [ ]:

class BERT4Rec(nn.Module):
    def __init__(self, vocab_size, num_depts, emb_dim, seq_len, num_seasons):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size + 1, emb_dim, padding_idx=vocab_size)
        self.pos_emb = nn.Embedding(seq_len, emb_dim)
        self.season_pos_emb = nn.Embedding(num_seasons, emb_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=4, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        self.struct_proj = nn.Sequential(
            nn.Linear(emb_dim * 4, emb_dim),
            nn.ReLU()
        )

        self.head = nn.Sequential(
            nn.Linear(emb_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, num_depts),
            nn.Sigmoid()
        )

    def forward(self, seqs, season_ids, customer_emb, product_emb, season_emb, weather_emb, promo_emb, decay=None):
        B, T = seqs.shape
        pos_ids = torch.arange(T, device=seqs.device).unsqueeze(0).expand(B, T)

        x = self.token_emb(seqs) + self.season_pos_emb(season_ids) + self.pos_emb(pos_ids)
        x = self.transformer(x)

        if decay is not None:
            x = (x * decay.unsqueeze(-1)).sum(1)  # Weighted sum
        else:
            x = x[:, 0, :]  # CLS token approach

        struct = torch.cat([season_emb, weather_emb, promo_emb, customer_emb], dim=-1)
        struct = self.struct_proj(struct)

        final = torch.cat([x, struct], dim=-1)
        return self.head(final)
